##### Copyright 2023 The MediaPipe Authors. All Rights Reserved.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
from zipfile import ZipFile
file_name = "..."

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
# Check image labels.
import os
image_path = "/home/tess/Documents/HAM10000_all_raw_images"
labels = []
for i in os.listdir(image_path):
  if os.path.isdir(os.path.join(image_path, i)):
    labels.append(i)
print(labels)

## Making a New Model



In [ ]:
!pip install mediapipe-model-maker

In [ ]:
# Import the necessary modules.
from mediapipe_model_maker import image_classifier

In [ ]:
# Split the archive into training, validation and test dataset.
data = image_classifier.Dataset.from_folder("HAM10000_all_raw_images")
train_data, remaining_data = data.split(0.8)
test_data, validation_data = remaining_data.split(0.5)

In [ ]:
# Retrain model.
spec = image_classifier.SupportedModels.MOBILENET_V2
#spec = image_classifier.SupportedModels.EFFICIENTNET_LITE0
#spec = image_classifier.SupportedModels.EFFICIENTNET_LITE2
#spec = image_classifier.SupportedModels.EFFICIENTNET_LITE4
hparams = image_classifier.HParams(export_dir="exported_model")
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

In [ ]:
# Run retraining.
model = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

In [ ]:
# Loss and accuracy metrics.
loss, acc = model.evaluate(test_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

In [ ]:
# F1 score, precision, and recall metrics.
import tensorflow as tf

metric_functions = model._metric_functions + [tf.keras.metrics.F1Score(),
 tf.keras.metrics.Precision(),
 tf.keras.metrics.Recall()]

model._model.compile(
    optimizer=model._optimizer,
    loss=model._loss_function,
    metrics=metric_functions,
 )

metrics = model.evaluate(test_data)
print(metrics)

In [ ]:
# Export the model bundle.
model.export_model()

# Rename the file to be more descriptive.
!mv exported_model/model.tflite skin_lesion_classifier.tflite

In [ ]:
from google.colab import files
files.download("/content/skin_lesion_classifier.tflite")